## Graph dependency


In [1]:
# import essentials libraries
# 
import nltk
from nltk.corpus import stopwords
import numpy as np 
import os 
import pandas as pd 
import re
import spacy
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd 
from nltk.corpus import stopwords 
from wordcloud import WordCloud, STOPWORDS
import math 
import statistics 
from sklearn.feature_extraction.text import CountVectorizer 
from textblob import TextBlob
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
import os 
import pandas as pd
from transformers import pipeline
import torch
import time
from datetime import timedelta
import psutil

import re

[nltk_data] Downloading package stopwords to C:\Users\Pin
[nltk_data]     Nean\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Pin
[nltk_data]     Nean\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1. concatenate all reviews from all sources (youtube, ebay, productreviews)

In [2]:
# files in the same repository 

data_folder = "data" 
data_filename = "product_review_heater4.csv"
data_filenames = ["product_review_heater1.csv","product_review_heater2.csv","product_review_heater3.csv","product_review_heater4.csv"]
dfs = []

for data_filename in data_filenames: 
    data_filepath = os.path.join(data_folder, data_filename)
    df = pd.read_csv(data_filepath)
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df['Reviews'] = df['Reviews'].dropna().apply(lambda s: eval(s))
    df = df.dropna()
    dfs.append(df)
    
df_pr = pd.concat(dfs)


In [3]:
data_folder = "support\heater_appliance"
data_filename = "combined_data.pkl"
data_filepath = os.path.join(data_folder, data_filename)

df_yt = pd.DataFrame(pd.read_pickle(data_filepath))
df_yt.columns = ["Title", "Page", "Reviews"]


In [4]:
data_folder = "support\heater_product"
data_filename = "combined_data.pkl"
data_filepath = os.path.join(data_folder, data_filename)

df_yt2 = pd.DataFrame(pd.read_pickle(data_filepath))
df_yt2.columns = ["Title", "Page", "Reviews"]

In [5]:
data_folder = "heater_data"
# data_filename = "product_review_heater4.csv"
data_filenames = os.listdir(data_folder)
dfs = []

for data_filename in data_filenames: 
    data_filepath = os.path.join(data_folder, data_filename)
    df = pd.read_csv(data_filepath)
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df['Reviews'] = df['Reviews'].dropna().apply(lambda s: eval(s))
    df = df.dropna(subset=['Reviews'])
    dfs.append(df)
    


    
df_uncleaned = pd.concat(dfs)
df_uncleaned.columns = map(str.strip,df_uncleaned.columns)
df_uncleaned = df_uncleaned.drop_duplicates(subset=['Title'], keep='last')


In [6]:
df_ebay = df_uncleaned[["Title", "Price", "Reviews"]]


In [7]:
df = pd.concat([df_pr, df_yt, df_yt2, df_ebay])

In [9]:
# create graph dependency function with spacy
import spacy

nlp=spacy.load('en_core_web_sm')

def graph_dependency_spec_gen(texts_list):
    numerical_specifications = []
    qualitative_specifications = []

    for j, text in enumerate(texts_list): 
        doc = nlp(str(text))
        i = 0 
        while i < len(doc): 
            token = doc[i]
            spec = []
            if token.dep_ == "nummod" or token.dep_ == "amod": 
                flag = token.dep_ == "nummod"

                while not token.dep_ in ["dobj", "pobj", "appos"] : 
    #                 print(f"{token.text:{8}} {token.dep_+' =>':{10}}   {token.head.text:{9}}  {spacy.explain(token.dep_)} ")
                    if token.text not in en_stops: 
                        spec.append(token.text)
                    i += 1
                    if i >= len(doc): 
                        break
                    token = doc[i]
    #             print(f"{token.text:{8}} {token.dep_+' =>':{10}}   {token.head.text:{9}}  {spacy.explain(token.dep_)} ")
                spec.append(token.text)

                if flag: 
                    numerical_specifications.append([spec,str(text)])
                else: 
                    qualitative_specifications.append([spec,str(text)])

            else: 
                i+=1
    return numerical_specifications, qualitative_specifications
         

In [11]:
df = df.explode('Reviews').dropna(subset=['Reviews'])

In [12]:
# create top words function

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.util import everygrams
from gensim.parsing.preprocessing import remove_stopwords , STOPWORDS
import itertools
from nltk.corpus import stopwords




# Create a Vectorizer Object
def topwords(doc): 
    en_stops = set(stopwords.words('english'))
    if type(doc[0]) == list: 
        doc = itertools.chain.from_iterable(doc)
    all_words = itertools.chain.from_iterable(map(lambda text: text.lower().split(), doc))
    all_words = filter(lambda word: word not in en_stops, all_words)

# vectorizer = CountVectorizer()

# vectorizer.fit(all_words) 
    counts = Counter(all_words)
    sorted_counts = sorted(counts, key=counts.get, reverse=True)
    return counts, sorted_counts
# Printing the identified Unique words along with their indices
# print("Vocabulary: ", sorted(vectorizer.vocabulary_, key=lambda x: vectorizer.vocabulary_.get(x)))
# vectorizer.vocabulary_['zloty']


In [13]:
en_stops = set(stopwords.words('english'))


In [14]:
# sentimentanalyser class 
class SentimentAnalyser: 
    def __init__(self, dataframes, text_column='Reviews'):
        df = dataframes
        df = df.explode(text_column)
        df['Reviews'] = df['Reviews'].apply(lambda t: self.preprocess_text(str(t)))
        self.df = df
        self.labels = {
            'text_column': text_column, 
            'sentiment_analysis': "sentiment"
        }

    def preprocess_text(self, text):
        text = text.lower()                             # all lower case
        text = re.sub(r'\[.*?\]', ' ', text)            # remove text within [ ] (' ' instead of '')
        text = re.sub(r'\<.*?\>', ' ', text)            # remove text within < > (' ' instead of '')
        text = re.sub(r'http\S+', ' ', text)            # remove website ref http
        text = re.sub(r'www\S+', ' ', text)             # remove website ref www

        text = text.replace('€', 'euros')               # replace special character with words
        text = text.replace('£', 'gbp')                 # replace special character with words
        text = text.replace('$', 'dollar')              # replace special character with words
        text = text.replace('%', 'percent')             # replace special character with words
        text = text.replace('\n', ' ')                  # remove \n in text that has it

        text = text.replace('\'', '’')                  # standardise apostrophe
        text = text.replace('&#39;', '’')               # standardise apostrophe

        text = text.replace('’d', ' would')             # remove ’ (for would, should? could? had + PP?)
        text = text.replace('’s', ' is')                # remove ’ (for is, John's + N?)
        text = text.replace('’re', ' are')              # remove ’ (for are)
        text = text.replace('’ll', ' will')             # remove ’ (for will)
        text = text.replace('’ve', ' have')             # remove ’ (for have)
        text = text.replace('’m', ' am')                # remove ’ (for am)
        text = text.replace('can’t', 'can not')         # remove ’ (for can't)
        text = text.replace('won’t', 'will not')        # remove ’ (for won't)
        text = text.replace('n’t', ' not')              # remove ’ (for don't, doesn't)

        text = text.replace('’', ' ')                   # remove apostrophe (in general)
        text = text.replace('&quot;', ' ')              # remove quotation sign (in general)

        text = text.replace('cant', 'can not')          # typo 'can't' (note that cant is a proper word)
        text = text.replace('dont', 'do not')           # typo 'don't'

        text = re.sub(r'[^a-zA-Z0-9]', r' ', text)      # only alphanumeric left
        text = text.replace("   ", ' ')                 # remove triple empty space
        text = text.replace("  ", ' ')                  # remove double empty space

        tokens = TreebankWordTokenizer().tokenize(text)
        tokens = map(WordNetLemmatizer().lemmatize, tokens)
        preprocessed_text = " ".join(tokens)

        return preprocessed_text
        
    def Textblob(self): 
        df = self.df.copy()
        df[self.labels['sentiment_analysis']] = df[self.labels['text_column']].apply(lambda text: TextBlob(text).sentiment.polarity)
        return df 

    def BERTSentimentAnalysis(self): 
        df = self.df.copy() 
        classifier = pipeline("sentiment-analysis") 
        df[self.labels['sentiment_analysis']] = df[self.text_column].apply(lambda text: classifier(text)[0]['label'])
        return df 
        
    def ZeroShot(self, candidates=["sustainable", "unsustainable"], use_gpu=False): 
        if use_gpu: 
            device_idx = 0
        else: 
            device_idx = -1
        df = self.df_combined.copy() 
        # classifier = pipeline("zero-shot-classification", device=0)
        classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=device_idx)
        df[self.labels['zero_shot']] = df[self.text_column].apply(lambda text: classifier(text, candidate_labels=candidates)[0]['label']) 
        return df

    def combine_text(self,list_of_text):                     # define combine_text to take (list_of_text)
        combined_text = ' '.join(list_of_text)          # do this
        return combined_text  

    def get_topwords(self,data,dataframesgroupby_model=True): 

        extra_stopwords = ["wa", "doe", "ha", "video", "one", "subscribe", "channel", "watch", "watching", "thanks", "thank"] 
        all_stopwords = stopwords.words('english')
        all_stopwords.extend(extra_stopwords) 
        STOPWORDS.update(all_stopwords)

        '''create dtm'''
        cv = CountVectorizer(stop_words=all_stopwords)                       # initialise cv w/o stopwords
        data_cv = cv.fit_transform(data)                      # apply cv
        dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names(), index=pd.Series(self.df.iloc[:]))
        

        return dtm
    

In [15]:
results = [['btu', 'emission output', 'carbon dioxide equivalent', 'water conservation', 'gallon per minute', 'material reuse', 'source', 'wind', 'power', 'thermal mass', 'fuel', 'appliance', 'space'],
['timer', 'behavioral property', 'control', 'feedback', 'temperature', 'furnace', 'hot air', 'interior', 'home.'],
['solar powered', 'electric resistance', 'emission', 'heat pump', 'geothermal system', 'size', 'space', 'central location', 'evenly heat', 'wall', 'ceiling', 'drapery', 'window', 'door closed', 'space', 'climate', 'building material', 'factor', 'size', 'capacity', 'location'],
['solar powered', 'electric resistance', 'heat pumps', 'upfront cost', 'operating cost', 'maintenance cost']]

open_ai_keywords = []

for result in results: 
    open_ai_keywords += result


In [16]:
from transformers import pipeline 

sentiment_classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


2. Identify with keywords are mentioned the most + sorting

In [17]:
pos_counts = []
neg_counts = []

for search_term in open_ai_keywords: 
    pos_count = 0 
    neg_count = 0 
    for comment in df['Reviews'].to_list():     
        if search_term in comment: 
#             print(comment)
#             print("\n")
#             print(sentiment_classifier(comment))
            try:
                if sentiment_classifier(comment)[0]['label'] == 'POSITIVE': 
                    pos_count += 1 
                else: 
                    neg_count += 1
            except: 
                pass
    pos_counts.append(pos_count)
    neg_counts.append(neg_count)
    # print("search term: ", search_term)
    # print("positive: ", pos_count)
    # print("negative: ", neg_count)
    # print("\n")

Token indices sequence length is longer than the specified maximum sequence length for this model (879 > 512). Running this sequence through the model will result in indexing errors


In [19]:
sentiment_df = pd.DataFrame({
    "search term": open_ai_keywords, 
    "positive": pos_counts,
    "negative": neg_counts,
})

In [20]:
sentiment_df.drop_duplicates()

,search term,positive,negative
0,btu,11,21
1,emission output,0,0
2,carbon dioxide equivalent,0,0
3,water conservation,0,0
4,gallon per minute,0,0
5,material reuse,0,0
6,source,82,120
7,wind,109,91
8,power,455,459
9,thermal mass,0,0


In [21]:
# sort keywords mentioned by most positive 
# sort keywords mentioned by most negative
# sort keywards that has most negative over positive ratio

rank_positive =sentiment_df.sort_values(by='positive', ascending=False).drop_duplicates()
rank_negative =sentiment_df.sort_values(by='negative', ascending=False).drop_duplicates()
sentiment_df['neg pos ratio'] = sentiment_df['negative'] / sentiment_df['positive']
most_pain = sentiment_df.sort_values(by='neg pos ratio', ascending=False).drop_duplicates()



In [325]:
most_pain.drop_duplicates().head(30)

,search term,positive,negative,neg pos ratio
11,appliance,11,48,4.363636
35,door closed,2,6,3.000000
39,factor,12,36,3.000000
37,climate,4,12,3.000000
47,operating cost,1,2,2.000000
45,heat pumps,3,6,2.000000
0,btu,11,21,1.909091
6,source,82,120,1.463415
19,hot air,20,29,1.450000
25,heat pump,52,57,1.096154


In [255]:
# take the top 20 most mention keywords from most_pain

top_pain = most_pain[['search term','neg pos ratio']].head(20)
top_pain_words = top_pain['search term'].tolist()
print('top pain words: ', top_pain_words)
top_neg_words = rank_negative['search term'].tolist()[:20]
print('top negegative words: ', top_neg_words)



top pain words:  ['appliance', 'door closed', 'factor', 'climate', 'operating cost', 'heat pumps', 'btu', 'source', 'hot air', 'heat pump', 'wall', 'power', 'wind', 'location', 'ceiling', 'window', 'fuel', 'interior', 'timer', 'temperature']
top negegative words:  ['power', 'temperature', 'space', 'wall', 'control', 'size', 'source', 'timer', 'furnace', 'wind', 'ceiling', 'window', 'heat pump', 'fuel', 'appliance', 'factor', 'home.', 'location', 'hot air', 'btu']


In [25]:
def preprocess_text( text):
       text = text.lower()                             # all lower case
       text = re.sub(r'\[.*?\]', ' ', text)            # remove text within [ ] (' ' instead of '')
       text = re.sub(r'\<.*?\>', ' ', text)            # remove text within < > (' ' instead of '')
       text = re.sub(r'http\S+', ' ', text)            # remove website ref http
       text = re.sub(r'www\S+', ' ', text)             # remove website ref www

       text = text.replace('€', 'euros')               # replace special character with words
       text = text.replace('£', 'gbp')                 # replace special character with words
       text = text.replace('$', 'dollar')              # replace special character with words
       text = text.replace('%', 'percent')             # replace special character with words
       text = text.replace('\n', ' ')                  # remove \n in text that has it

       text = text.replace('\'', '’')                  # standardise apostrophe
       text = text.replace('&#39;', '’')               # standardise apostrophe

       text = text.replace('’d', ' would')             # remove ’ (for would, should? could? had + PP?)
       text = text.replace('’s', ' is')                # remove ’ (for is, John's + N?)
       text = text.replace('’re', ' are')              # remove ’ (for are)
       text = text.replace('’ll', ' will')             # remove ’ (for will)
       text = text.replace('’ve', ' have')             # remove ’ (for have)
       text = text.replace('’m', ' am')                # remove ’ (for am)
       text = text.replace('can’t', 'can not')         # remove ’ (for can't)
       text = text.replace('won’t', 'will not')        # remove ’ (for won't)
       text = text.replace('n’t', ' not')              # remove ’ (for don't, doesn't)

       text = text.replace('’', ' ')                   # remove apostrophe (in general)
       text = text.replace('&quot;', ' ')              # remove quotation sign (in general)

       text = text.replace('cant', 'can not')          # typo 'can't' (note that cant is a proper word)
       text = text.replace('dont', 'do not')           # typo 'don't'

       text = re.sub(r'[^a-zA-Z0-9]', r' ', text)      # only alphanumeric left
       text = text.replace("   ", ' ')                 # remove triple empty space
       text = text.replace("  ", ' ')                  # remove double empty space

       tokens = TreebankWordTokenizer().tokenize(text)
       tokens = list(map(WordNetLemmatizer().lemmatize, tokens))
       preprocessed_text = " ".join(tokens)

       return preprocessed_text

In [26]:
# extract all reviews from the dataset

all_reviews = df['Reviews'].tolist()


In [27]:
# tokenize, leminizing (preprocessing) for top words
# do the same for the reviews dataset

top_pain_key =[]
for word in top_pain_words: 
    top_pain_key.append(preprocess_text(word))


all_reviews_clean =[]
for review in all_reviews:
    all_reviews_clean.append(preprocess_text(review))


In [118]:
# for all the cleaned reviews, check if the top pain keywords are present, if it is, then store it for later
# same for positive keywords
# save both original and cleaned reviews
pain_comments = {}
pain_comments_original = {}

i = 0

for all_review in all_reviews_clean: 
    for neg_key in top_pain_key: 
        if neg_key in all_review: 
            if neg_key not in pain_comments: 
                pain_comments[neg_key] = [all_review]
                pain_comments_original[neg_key] = [all_reviews[i]]
            else:
                if all_review not in pain_comments[neg_key]: 
                    pain_comments[neg_key].append(all_review)
                    pain_comments_original[neg_key].append(all_reviews[i])
    i += 1

                

In [123]:
# checking 
print(pain_comments['btu'][12])
print(pain_comments_original['btu'][12])


bought this heater to help supplement heat in our large informal office we are presently using an 80 000 btu kerosene heater but the fume were bothering u when it shuts off other than that it warmed the large office space very quickly and evenly we wanted to use the dyna glo because it is propane hoping there will be far le fume while we were attempting to get the heater to stay lit it wa producing heat but not nearly a much a the kero 80k would nor did the fan push the heat throughout the office space a fast this is only 20k btu le but it felt far le than that we had to return the dyna glo maybe we will get another one or something a bit better and pay a few dollar more
Bought this heater to help supplement heat in our large informal office. We are presently using an 80,000 BTU kerosene heater but the fumes were bothering us when it shuts off other than that it warmed the large office space very quickly and evenly.We wanted to use the Dyna Glo because it is propane hoping there will b

In [29]:
# pain_comments

3. Sentiment Analysis

In [30]:
# for each comment in the stored list, do sentiment analysis, and store the result (score and label together with the comment)

score ={}

for key in pain_comments:
    score[key] = []
    for review in pain_comments[key]:
        try:
            sc = sentiment_classifier(review)
            score[key].append([review,sc[0]['score'],sc[0]['label']])
        except: 
            pass




In [121]:
score['btu'][10]

['purchased this heater 14 year ago and used it every winter since then it is small but btu are btu regardless of size and it heat my cabin bathroom just fine in the winter mine doe not have the newer tip over safety switch i have never had any paint chipping issue and it look a new a it did the day i purchased it my only complaint is that the heater hi lo fan selector switch is very tight to turn but i have learned to live with it the cord is also long enough that i do not need an extension cord which make portable electrical heater even more unsafe hope this help with your decision',
 0.8231490850448608,
 'NEGATIVE']

In [35]:
#split the comments into positive and negative dictionary
# the key of the dict is the keywords, and the value is the (comment, score and label(positive/negative))

score_pos = {}
score_neg = {}

for key in score.keys():
    score_pos[key] = []
    score_neg[key] = []
    for d in score[key]:
        if d[2] == 'POSITIVE':
            score_pos[key].append(d)
        else:
            score_neg[key].append(d)



In [36]:
#sort all the list in descending order from the dictionary
#highest score (negative/positive first)

from operator import itemgetter

for k,v in score_neg.items():
    score_neg[k] = sorted(v, key=itemgetter(1), reverse=True)
for k,v in score_pos.items():
    score_pos[k] = sorted(v, key=itemgetter(1), reverse=True)
# score_neg

In [37]:
# make another dict that only store comments ( and make sure not repeated)
top_neg_all ={}
for k,v in score_neg.items():
    top_neg_all[k] = []
    for i in score_neg[k][:100]:
        if i[0] not in top_neg_all[k]:
           top_neg_all[k].append(i[0])

top_pos_all ={}
for k,v in score_pos.items():
    top_pos_all[k] = []
    for i in score_pos[k][:100]:
        if i[0] not in top_pos_all[k]:
           top_pos_all[k].append(i[0])


# top_neg_all

4.Graph dependency


In [40]:
# now we have all the top negative / positive comment in the order
# we run graph dependency on each comment 
# output is the numerical and qualitative data extract from those top positive/negative comments

gp_neg_all ={}

for k,v in top_neg_all.items():
    gp_neg_all[k] = graph_dependency_spec_gen(top_neg_all[k])
   

In [41]:
# remove keys without any value

to_pop = []
for k,v in gp_neg_all.items():
    if len(v) == 0:
        to_pop.append(k)

for i in to_pop:
    gp_neg_all.pop(i)
gp_neg_all.keys()



dict_keys(['heat pump', 'power', 'timer', 'temperature', 'wind', 'window', 'factor', 'operating cost', 'location', 'source', 'ceiling', 'door closed', 'wall', 'appliance', 'fuel', 'interior', 'climate', 'hot air', 'btu'])

In [271]:
# example of graph dependency extraction

print('Example of graph dependency results:')
print('1. appliances')
print(gp_neg_all['appliance'][0][4])
print( gp_neg_all['appliance'][0][5])
print(gp_neg_all['appliance'][0][18])
print('\n')
print('2. heat pump')
print(gp_neg_all['heat pump'][0][6])
print(gp_neg_all['heat pump'][0][1])
print(gp_neg_all['heat pump'][0][2])

# testing with ner
test_ner = [gp_neg_all['appliance'][0][5], gp_neg_all['appliance'][0][18], gp_neg_all['heat pump'][0][6], gp_neg_all['heat pump'][0][1], gp_neg_all['heat pump'][0][3]]

Example of graph dependency results:
1. appliances
[['2', 'day'], 'hi we had a df700 installed 3 year ago last thursday it stopped working we had a gas fitter come to the house and he said we had a new gas meter installed and they had forgotten to purge the meter charged u dollar275 it worked for 2 day and then stopped again there is a spark but no gas getting through the gas is working in the house a we have other appliance on gas the gas fitter say he think now it might be an electrical problem we do not know what to do do we spend hundred more dollar asking people to come fix it who do not really know why would it suddenly stop working completely after only 3 year please advise what we should do']
[['hundred', 'dollar'], 'hi we had a df700 installed 3 year ago last thursday it stopped working we had a gas fitter come to the house and he said we had a new gas meter installed and they had forgotten to purge the meter charged u dollar275 it worked for 2 day and then stopped again there

In [257]:
from pprint import pprint
import spacy 
from spacy import displacy
  
nlp = spacy.load('en_core_web_sm') 
  

 
for text in test_ner: 
    entities= nlp(text[1])
    #to print all the entities with iob tags 
    # pprint([ (X, X.ent_iob_, X.ent_type_) for X in entities] )
 
    #to print just named entities use this code 
    print("\nNamed entities in this text are")
    # for ent in entities.ents: 
    #     print(ent.text,ent.label_)
 
    # visualize named entities  
    displacy.render(entities, style='ent', jupyter=True)


Named entities in this text are



Named entities in this text are



Named entities in this text are



Named entities in this text are



Named entities in this text are


5.OpenAI keywords extraction and summarization

In [68]:
# get qualitative respone from OpenAI
b = gp_neg_all['heat pump']
quan_b_com = [i[1] for i in b[0]]
#print('all reviews that has quantitative data')
# print(quan_b_com)



In [222]:
import os
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt= "Extract keywords from this text: "+ '.'.join(quan_b_com[:20]) +" ",
  temperature=1,
  max_tokens=100,
  top_p=1.0,
  frequency_penalty=1.0,
  presence_penalty=1.0
)
result = response.choices[0].text.replace('\n', ' ')


'  Broken heat pump, expensive to fix, noisy, short warranty'

In [273]:
# Summarry for designing for sustainable heater
import os
import openai
import time

openai.api_key = os.getenv('OPENAI_API_KEY')

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt=  'heater, ' + '.'.join(quan_b_com[1:20]) + "\n\nTl;dr:\n\n", # the first 20 sentences of the comments
  temperature=0.3,
  max_tokens=500,
  top_p=1,
  frequency_penalty=1,
  presence_penalty=1
)
e= response.choices[0]['text'].replace("\n", "")
e

'1. Sanden heat pumps are extremely noisy and overpriced.2. They only come with a 12 month warranty, and their service fees and call-out charges are ridiculous.3. Our experience with Sanden has not been great – the heat pump gave out after 4 1/2 years, and it was out of warranty. The cost to replace it was $4,000, and we did not replace it with another heat pump because we are not prepared to spend that much money on a product that should have had a much better life expectancy.'

6. Getting product specification


In [212]:
# attempt to get numerical product specification
# 1. British Thermal Unit
value = []
for i in score['btu']:
    entities= nlp(i[0])
    for i in range(len(entities.ents)):
        if 'btu' in entities.ents[i].text or 'k' in entities.ents[i].text:
           value.append(entities.ents[i].text)

        if entities.ents[i].label_ == 'CARDINAL':
            x = entities.ents[i].text.replace(' ', '')
            if x.isnumeric():
               
                # convert to digit 
                if len(entities.ents[i].text) > 2:
                   value.append(entities.ents[i].text)
                   
value_clean = []           
print(value)
for i in value:
    if '000' in i:
        i = i.replace(' ', '')
    if i.isnumeric():
        value_clean.append(i)
    else:
        for j in range(len(i)):
            if i[j] == 'k':
                if i[j-1].isnumeric():
                    k = 0 
                    for k in range(j-1):
                        if i[k] == ' ':
                            value_clean.append(i[k+1:j]+'000')

value_clean=list(map(int,value_clean))
print(list(value_clean))



['4000', '9000', '18 000', '6 18k', '6 000', '18 000', '80 000', '630', '80 000', '10 000', '18k', '60 000', '110k', 'this weekend', '150k btu', '30 000', '30 000', '50 000', '80k']
[4000, 9000, 18000, 18000, 6000, 18000, 80000, 630, 80000, 10000, 60000, 30000, 30000, 50000]


In [275]:
from statistics import mean, median,quantiles
from numpy import quantile
btu_pos = 11
btu_neg = 21
btu_total = btu_pos + btu_neg

positivity = btu_pos/btu_total

print('statistic for btu')
print('positivity rate (positive count / negative count): ', positivity)
print('mean: ', mean(value_clean))
print(f'The positivity rate {positivity}, that yields a btu value of: ', pd.DataFrame(value_clean).quantile(positivity).values[0], f' at quantile {positivity}')
if positivity < 0.5:
    print(f'The positivity rate is lower than 50% and the mean is {mean(value_clean)} > {pd.DataFrame(value_clean).quantile(positivity).values[0]}, so the btu value is at minimum ' ,pd.DataFrame(value_clean).quantile(positivity).values[0])



        

statistic for btu
positivity rate (positive count / negative count):  0.34375
mean:  29545
The positivity rate 0.34375, that yields a btu value of:  13750.0  at quantile 0.34375
The positivity rate is lower than 50% and the mean is 29545 > 13750.0, so the btu value is at minimum  13750.0


In [252]:
# get qualitative product sepefication data from OpenAI
import os
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt=  "Extract keywords from this text: " + ' btu , ' + '.'.join([i[0] for i in score['btu']]),
  temperature=1,
  max_tokens=250,
  top_p=0.5,
  frequency_penalty=1,
  presence_penalty=1
)
e= response.choices[0]['text'].replace("\n", "")
e

'-BTU -Watts -Heaters -Propane -Pilot light -Two heat settings -Mr. Heater -Dyna Glo'

In [309]:
# 2. Temperature
value_temp = []
for i in score['temperature']:
    entities= nlp(i[0])
    for i in range(len(entities.ents)):
        if 'c' in entities.ents[i].text :
            value_temp.append(entities.ents[i].text)
        
        if 'degree' in entities.ents[i].text:
            
            x =entities.ents[i].text.replace(' ', '')
            
            for j in range(len(x)):
                if x[j].isnumeric():
                    pass
                else:
                    value_temp.append(x[:j])
                    break     
                   
value_temp_clean = []           
print(value_temp)
for i in value_temp:
    x = re.findall('\d+', i )
    if len(x) > 0:
        if int(x[0]) < 120:
            value_temp_clean.append(x[0]) 
    
value_temp_clean=list(map(int,value_temp_clean))
print(list(value_temp_clean))





['66c', '50', '', '65', '50', 'second', '5', 'canadian', '21', 'canada', 'december', '50', 'no le than 60c', 'second', 'chilly morning', '0c', '50', '5c', 'oct 2020', '50c', '5 10 second', 'second', '21', 'second', '60', '60', '50', 'patrick', '27', '30', '65', '86', '10', '510', 'second', 'second', '40', '60', '41', '41', '200', 'this coming weekend', '58', 'eco', '68', '60', '85', 'american', '18', '37 degree a couple night', '37', '40', '6870', 'michigan', '6368', '72', '55', 'cold day', '70', '30', 'a couple of hour', '40', 'california', '105c', 'chicago', 'second', '10', '8', '80', '70', '30', 'x6 x3 coop', '', '10', '30', '64', '30', '20', '32', '70', 'american']
[66, 50, 65, 50, 5, 21, 50, 60, 0, 50, 5, 50, 5, 21, 60, 60, 50, 27, 30, 65, 86, 10, 40, 60, 41, 41, 58, 68, 60, 85, 18, 37, 37, 40, 72, 55, 70, 30, 40, 105, 10, 8, 80, 70, 30, 6, 10, 30, 64, 30, 20, 32, 70]


In [319]:
temp_pos = 565
temp_neg = 333
temp_total = temp_pos + temp_neg

positivity_temp = round(temp_pos/temp_total,2)

print('statistic for temperature')
print('positivity rate (positive count / negative count): ', positivity_temp)
print('mean: ', round(mean(value_temp_clean),2))
temp = round(pd.DataFrame(value_temp_clean).quantile(positivity_temp).values[0],2)
print(f'The positivity rate {positivity_temp}, that yields a temperature value of: ', temp, f' at quantile {positivity_temp}')
if positivity_temp > 0.5:
    print(f'The positivity rate is higher than 50% and the mean is {round(mean(value_temp_clean),2)} < {temp}, so the temperatuer value should be above ' ,temp)


statistic for temperature
positivity rate (positive count / negative count):  0.63
mean:  43.45
The positivity rate 0.63, that yields a btu value of:  53.8  at quantile 0.63
The positivity rate is higher than 50% and the mean is 43.45 < 53.8, so the temperatuer value should be above  53.8


In [324]:
response = openai.Completion.create(
  engine="text-davinci-002",
  prompt=  "Extract keywords from this text: " + ' btu , ' + '.'.join([i[0] for i in score['temperature']][:30]),
  temperature=1,
  max_tokens=250,
  top_p=0.5,
  frequency_penalty=1,
  presence_penalty=1
)
t= response.choices[0]['text'].replace("\n", "")
t

'.-Btu -Evoheat 270 -Hot water within an hour -Inbuilt timer -Solar power -Peak temperature control -Vacation mode -Same day hot water in Sydney -Thermostat '